<a href="https://colab.research.google.com/github/FAID-Nacera/AutoML/blob/main/H2O_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#============================
# Step 1: Install packages
#============================

In [2]:
pip install h2o

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 MB 5.1 MB/s eta 0:00:00


In [3]:
pip install h2o auto-sklearn tpot mljar-supervised

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.3/127.3 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 41.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for detai

In [4]:
pip install pandas numpy scikit-learn matplotlib

In [5]:
#=================================
# Step 2: Import libraries
#=================================

In [6]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

print("=== AutoML Demo with H2O ===")

=== AutoML Demo with H2O ===


In [7]:
#=================================
# Step 3: Load YOUR dataset
#=================================

In [8]:
print("1. Loading real dataset...")
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("homework.csv")
df = df.drop(columns=["rownames"])
df = df.replace({"yes": 1, "no": 0})

X = df.drop(columns=["price"])
y = "price"

print(f"Dataset shape: {df.shape}")
print("First 5 rows:")
print(df.head())

1. Loading real dataset...


Saving homework.csv to homework.csv
Dataset shape: (546, 12)
First 5 rows:
   price  lotsize  bedrooms  bathrms  stories  driveway  recroom  fullbase  \
0  42000     5850         3        1        2         1        0         1   
1  38500     4000         2        1        1         1        0         0   
2  49500     3060         3        1        1         1        0         0   
3  60500     6650         3        1        2         1        1         0   
4  61000     6360         2        1        1         1        0         0   

   gashw  airco  garagepl  prefarea  
0      0      0         1         0  
1      0      0         0         0  
2      0      0         0         0  
3      0      0         0         0  
4      0      0         0         0  


/tmp/ipython-input-2090404708.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"yes": 1, "no": 0})


In [9]:
#=================================
# Step 4: Initialize H2O
#=================================

In [10]:
print("\n2. Initializing H2O...")
h2o.init()


2. Initializing H2O...
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.17" 2025-10-21; OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04); OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.12/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnozlcp_g
  JVM stdout: /tmp/tmpnozlcp_g/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpnozlcp_g/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_unknownUser_b5i8kq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.168 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [11]:
# Convert pandas DataFrame to H2O Frame
h2o_df = h2o.H2OFrame(df)

# Features for AutoML
x = h2o_df.columns
x.remove(y)

print(f"Features: {len(x)}")
print(f"Target: {y}")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Features: 11
Target: price


In [12]:
#=================================
# Step 5: Split the data
#=================================

In [13]:
train, test = h2o_df.split_frame(ratios=[0.8], seed=42)
print(f"Training set: {train.shape[0]} rows")
print(f"Test set: {test.shape[0]} rows")

Training set: 426 rows
Test set: 120 rows


In [14]:
#=================================
# Step 6: Run AutoML (Regression)
#=================================

In [15]:
print("\n3. Running AutoML ...")

aml = H2OAutoML(
    max_models=10,          # Maximum number of models
    seed=42,                # Reproducibility
    max_runtime_secs=300,   # 5 minutes max runtime
    exclude_algos=["GLM"],  # Optional, to speed up
    verbosity="info"
)

aml.train(x=x, y=y, training_frame=train)


3. Running AutoML ...
AutoML progress: |
23:56:17.138: Project: AutoML_1_20260107_235617
23:56:17.141: 5-fold cross-validation will be used.
23:56:17.142: Setting stopping tolerance adaptively based on the training frame: 0.04845015831115092
23:56:17.142: Build control seed: 42
23:56:17.144: training frame: Frame key: AutoML_1_20260107_235617_training_py_2_sid_902c    cols: 12    rows: 426  chunks: 1    size: 5402  checksum: -3487615980247602864
23:56:17.144: validation frame: NULL
23:56:17.146: leaderboard frame: NULL
23:56:17.146: blending frame: NULL
23:56:17.146: response column: price
23:56:17.146: fold column: null
23:56:17.147: weights column: null
23:56:17.180: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealin

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_1_20260107_235617


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    43                 43                          15262                  6            10           8.69768       17            28            23.4186

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 129829739.45104937
RMSE: 11394.28538571197
MAE: 8173.490211450998
RMSLE: 0.16385477098738063
Mean Residual Deviance: 129829739.45104937

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 263875250.5627776
RMSE: 16244.237457103907
MAE: 11740.728812524536
RMSLE: 0.22802604993182624
Mean Residual Deviance: 263875250.5627776

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan          0            nan           nan           nan           nan           nan
loglikelihood           nan          0            nan           nan           nan           nan           nan
mae                     11840.8      598.469      11744         12243.8       11890.2       12436         10889.9
mean_residual_deviance  2.68972e+08  2.89476e+07  2.60038e+08   2.85457e+08   3.05914e+08   2.64696e+08   2.28757e+08
mse                     2.68972e+08  2.89476e+07  2.60038e+08   2.85457e+08   3.05914e+08   2.64696e+08   2.28757e+08
r2                      0.620948     0.0188435    0.59306       0.619592      0.62072       0.645841      0.62553
residual_deviance       2.68972e+08  2.89476e+07  2.60038e+08   2.85457e+08   3.05914e+08   2.64696e+08   2.28757e+08
rmse                    16381.1      887.602      16125.7       16895.5       17490.4       16269.5       15124.7
rmsle                   0.229545     0.0149485    0.229393      0.244072      0.218727      0.244489      0.211041

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2026-01-07 23:56:31  0.820 sec   0                  26673.4          20231.4         7.11468e+08
    2026-01-07 23:56:31  0.838 sec   5                  21160.7          15541.9         4.47776e+08
    2026-01-07 23:56:31  0.851 sec   10                 17480.7          12669           3.05576e+08
    2026-01-07 23:56:31  0.863 sec   15                 15195.3          10790.4         2.30898e+08
    2026-01-07 23:56:31  0.877 sec   20                 13867.3          9726.38         1.92303e+08
    2026-01-07 23:56:31  0.890 sec   25                 12938.7          9171.15         1.67411e+08
    2026-01-07 23:56:31  0.905 sec   30                 12377.5          8817.84         1.53203e+08
    2026-01-07 23:56:31  0.922 sec   35                 11942.8          8526.12         1.4263e+08
    2026-01-07 23:56:31  0.938 sec   40                 11558.8          8295.76         1.33606e+08
    2026-01-07 23:56:31  0.947 sec   43                 11394.3          8173.49         1.2983e+08

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
lotsize     3.8068e+11             1                    0.310628
bathrms     2.01416e+11            0.529095             0.164352
airco       1.7407e+11             0.457262             0.142038
garagepl    1.25283e+11            0.329104             0.102229
stories     9.14

In [16]:
#=================================
# Step 7: Display results
#=================================

In [17]:
print("\nLeaderboard (Top Models):")
lb = aml.leaderboard
print(lb.head())

# Best model
best_model = aml.leader
print(f"\nBest Model: {best_model.model_id}")


Leaderboard (Top Models):
model_id                                                    rmse          mse      mae     rmsle    mean_residual_deviance
GBM_4_AutoML_1_20260107_235617                           16244.2  2.63875e+08  11740.7  0.228026               2.63875e+08
StackedEnsemble_AllModels_1_AutoML_1_20260107_235617     16253.5  2.64175e+08  11711.8  0.226736               2.64175e+08
StackedEnsemble_BestOfFamily_1_AutoML_1_20260107_235617  16263    2.64486e+08  11688.6  0.227118               2.64486e+08
GBM_3_AutoML_1_20260107_235617                           16492.5  2.72003e+08  11986.2  0.229443               2.72003e+08
GBM_2_AutoML_1_20260107_235617                           16661.7  2.77611e+08  12084.3  0.232205               2.77611e+08
DRF_1_AutoML_1_20260107_235617                           16661.9  2.77618e+08  11975    0.233276               2.77618e+08
GBM_5_AutoML_1_20260107_235617                           16688.1  2.78494e+08  11964.9  0.232531               2

In [18]:
#=================================
# Step 8: Make predictions
#=================================

In [19]:
predictions = best_model.predict(test)
print("\nPredictions sample:")
print(predictions.head())


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%

Predictions sample:
  predict
  96828.8
  41812.4
  65499.2
  45191.3
  74650.5
  44200.3
  45893.4
  34278.1
  59102
  56408.2
[10 rows x 1 column]



In [20]:
#=================================
# Step 9: Model performance
#=================================

In [21]:
performance = best_model.model_performance(test)
print("\nModel Performance:")
print(performance)


Model Performance:
ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 234757107.9239343
RMSE: 15321.785402619837
MAE: 11724.160485567585
RMSLE: 0.2329177392841426
Mean Residual Deviance: 234757107.9239343


In [22]:
#=================================
# Step 10: Shutdown H2O
#=================================

In [23]:
print("\nShutting down H2O...")
h2o.cluster().shutdown()

print("\n=== AutoML Complete ===")


Shutting down H2O...
H2O session _sid_902c closed.

=== AutoML Complete ===
